# SQL DATA ANALYSIS TASK

# Context
Fenix is a company that is in renewable energy. To this end, it sells solar kits to populations who are in need. Customers have the option of paying for the kit in cash or on credit. If the customer chooses to buy on credit, he can choose between one year (360 days) or two years (720). He is then asked to pay an amount per day (daily_rate), this amount depending on the duration of the loan and the type of kit he has chosen and corresponds to a day of light. So every time he pays that amount, he has a day of light. After 24 hours the system hangs and it has to make another payment to take advantage of the light. He can decide to pay for more than one day of light. The day number is then the whole part of the amount / daily rate ratio. After this number of days, the system still locks. When a customer spends more than 30 days without light, he is considered inactive. Paid vs Expected (PvE) means the ratio between the total amount paid by the client and the amount he should have paid up to date. It allows to appreciate the recovery.

Let's consider the following tables

Loan_current_daitails

loan_id	|customer_id|daily_rate|days_elapsed|Days_til_lock_out|Total_paid
--------|-----------|----------|------------|-----------------|----------
loan_id_1|customer_id_1|daily_rate_1|days_elapsed_1|Days_til_lock_out_1|Total_paid_1
loan_id_2|customer_id_2|daily_rate_2|days_elapsed_2|Days_til_lock_out_2|Total_paid_2
.........|.............|............|..............|...................|............
loan_id_n|customer_id_n	|daily_rate_n|days_elapsed_n|Days_til_lock_out_n|Total_paid_n


Person_democraphic

customer_id	|country|	region|	profession
------------|-------|---------|-----------
customer_id_1|	Country_1	|Region_1	|profession_1
.............|	............|..........	|............
customer_id_j|	Country_j|	Region_j	|profession_j


Sales_details

loan_id	|Product_type	|Date_fullfiled	|loan_duration
--------|----------------|--------------|-------------
loan_id_1	|Product_type_1	|Date_fullfiled_1	|loan_duration_1
...|	...	|...	|...
loan_id_n|	Product_type_n|	Date_fullfiled_n|	loan_duration_n

### Tables description

* **Loan_id:** unique loan identifier

* **Customer_id:** unique customer identifier

* **Days_elapsed:** the number of days the client spends with us

* **Daily_rate:** expected amount from customer per day

* **Days_til_lock_out:** If positive, then corresponds to the number of days of light remaining before the system hangs. If negative, corresponds to the number of days since the system was blocked (the client has been without light since this number of days)

* **Total_paid:** total amount to be paid by the customer

* **Country:** Customer's country

* **Region:** customer department

* **Profession:** client's profession

* **Product_type:** the type of kit purchased by the customer

* **Loan_duration:** the duration of the customer's loan (360 or 720 days)

* **Date_fullfiled:** the date of the sale

### The datasets
The datasets exist in csv formats and can be downloaded and saved on disk

In [2]:
%load_ext sql
%sql sqlite:///challengedb.sqlite

In [126]:
%%sql
DROP TABLE IF EXISTS Loan_current_daitails;
DROP TABLE IF EXISTS Person_democraphic;
DROP TABLE IF EXISTS Sales_details;

 * sqlite:///challengedb.sqlite
Done.
Done.
Done.


[]

In [14]:
import pandas as pd
import sqlite3

conn = sqlite3.connect("challengedb.sqlite")
customers = pd.read_csv('Loan_current_daitails.csv')
orders = pd.read_csv('Person_democraphic.csv')
salesman = pd.read_csv('Sales_details.csv')

customers.to_sql("Loan_current_daitails", conn, index=False, if_exists="replace")
orders.to_sql("Person_democraphic", conn, index=False, if_exists="replace")
salesman.to_sql("Sales_details", conn, index=False, if_exists="replace")

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\generic.py:2127: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  dtype=dtype)


### Write a SQL code to determine the total number of loans on the one hand and the total number of customers on the other

In [128]:
%%sql

SELECT COUNT(*) total_num_loans, COUNT(DISTINCT(customer_id)) total_customers
FROM Loan_current_daitails;

 * sqlite:///challengedb.sqlite
Done.


total_num_loans,total_customers
5000,4979


### Write a SQL code to determine the total number of loans by type of kit

In [187]:
%%sql

SELECT s.Product_type, COUNT(*) num_of_loans
FROM Loan_current_daitails l
JOIN Sales_details s
ON s.loan_id = l.loan_id
GROUP BY 1;

 * sqlite:///challengedb.sqlite
Done.


product_type,num_of_loans
Fenix 2,841
Fenix 3,974
Fenix Power 2+ Tondeuse,18
Fenix Power 4 + Woofer BJ,35
Fenix Power TV 19in FTA BJ,14
"Fenix Power TV 19in FTA Upgrade, BJ",28
Fenix Power TV 24in FTA BJ,634
"Fenix Power TV 24in FTA Upgrade, BJ",177
Fenix Radio +3,1791
Fenix Radio +5,422


### Write SQL code to determine the number of inactive loans

In [119]:
%%sql

SELECT COUNT(*) Inactive_loans
FROM Loan_current_daitails
WHERE Days_til_lockout < -30;

 * sqlite:///challengedb.sqlite
Done.


Inactive_loans
626


### Write a SQL code to determine the regions where the total amount of payments exceeds 1,000,000

In [130]:
%%sql

SELECT p.region, SUM(l.Total_paid) total_amount
FROM Loan_current_daitails l, Person_democraphic p
WHERE l.customer_id = p.customer_id
GROUP BY 1
HAVING total_amount > 1000000;

 * sqlite:///challengedb.sqlite
Done.


region,total_amount
Alibori,8559985
Atacora,7802685
Atlantique,51293780
Borgou,18947292
Collines,11904427
Couffo,4812070
Donga,9936982
Littoral,3980519
Mono,11621621
Oueme,30027194


### Write a SQL code to classify by region and in ascending order, the loans according to the total amount paid

In [131]:
%%sql

SELECT p.region, SUM(l.Total_paid) total_amount_paid, COUNT(*) loans
FROM Loan_current_daitails l, Person_democraphic p
WHERE l.customer_id = p.customer_id
GROUP BY 1
ORDER BY 2;

 * sqlite:///challengedb.sqlite
Done.


region,total_amount_paid,loans
Littoral,3980519,88
Couffo,4812070,163
Atacora,7802685,183
Alibori,8559985,222
Zou,9540211,266
Plateau,9691206,273
Donga,9936982,258
Mono,11621621,385
Collines,11904427,444
Borgou,18947292,466



### Write a SQL code to determine the top 10 occupations by region with the highest total amount of payments

In [195]:
%%sql

SELECT  p.region, p.profession, SUM(l.Total_paid) total_paid
        FROM Loan_current_daitails l, Person_democraphic p
        WHERE l.customer_id = p.customer_id
        GROUP BY 1, 2
        ORDER BY 3 DESC;


 * sqlite:///challengedb.sqlite
Done.


region,Profession,total_paid
Atlantique,Teacher,6062410
Atlantique,Security agent,5587249
Atlantique,Driver,5457680
Atlantique,Nurse,5394546
Atlantique,Ingenieur,5302315
Atlantique,Farmer,5041556
Atlantique,Doctor,4916428
Atlantique,Tehnician,4790191
Atlantique,Cmmercial,4545867
Atlantique,Statistian,4195538


In [ ]:
%%sql

WITH abc AS (
        SELECT  p.region, p.profession, COUNT(p.Profession) count, SUM(l.Total_paid) total_paid
        FROM Loan_current_daitails l, Person_democraphic p
        WHERE l.customer_id = p.customer_id
        GROUP BY 1, 2
        ORDER BY 4
        ),
  bde AS (
        SELECT *, ROW_NUMBER () OVER(PARTITION BY region ORDER BY total_paid desc) rank 
        FROM abc
        )
SELECT region, profession, total_paid
FROM bde
WHERE rank = 1
ORDER BY total_paid desc